In [12]:
from modelgym.models import CtBClassifier
from modelgym.utils import XYCDataset
from modelgym.trainers import TpeTrainer
from modelgym.metrics import Accuracy
from modelgym.report import Report
from modelgym.utils import ModelSpace
from hyperopt import hp
import numpy as np
from sklearn.datasets import make_classification
from pathlib import Path

Wonderland optimization


In [13]:
from wonderclient import ModelGymClient


ip = "localhost:50051"
workingFolder = Path.home() / "repo-storage/test"
client = ModelGymClient('localhost:50051', workingFolder, "user")

In [14]:
n_features=20
X, y = make_classification(n_samples=500, n_features=n_features, n_informative=10, n_classes=2)
dataset = XYCDataset(X, y)
y = np.array([y])
data = np.concatenate((X, y.T), axis=1)
path_to_data = workingFolder / "DATA/tmp_data12345678.csv"
np.savetxt(path_to_data, data, 
           fmt='%.2f', 
           header=','.join([str(x) for x in range(n_features)]+['y']),
           delimiter=',')

In [15]:
models = ModelSpace(CtBClassifier,
                    space={'learning_rate': hp.loguniform('learning_rate', -5, -1)},
                    space_update=True)
trainer = TpeTrainer(models)

In [16]:
%%time
##Cluster optimization
trainer.crossval_optimize_params(Accuracy(), path_to_data, metrics=[Accuracy()], client=client, opt_evals=3)

CPU times: user 400 ms, sys: 28 ms, total: 428 ms
Wall time: 7min 30s


In [17]:
reporter = Report(trainer.get_best_results(), dataset, [Accuracy()])
reporter.print_all_metric_results()


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~    accuracy    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

                           tuned
CtBClassifier  0.914039 (+0.00%)


In [21]:
%%time
##Non-cluster optimization
models = ModelSpace(CtBClassifier,
                    space={'learning_rate': hp.loguniform('learning_rate', -5, -1)},
                    space_update=True)
trainer_2 = TpeTrainer(models)
trainer_2.crossval_optimize_params(Accuracy(), dataset, metrics=[Accuracy()], opt_evals=3)

CPU times: user 5min 52s, sys: 3min 32s, total: 9min 24s
Wall time: 4min 18s


In [22]:
reporter_2 = Report(trainer.get_best_results(), dataset, [Accuracy()])
reporter_2.print_all_metric_results()


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~    accuracy    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

                           tuned
CtBClassifier  0.914039 (+0.00%)
